## TFX - Deploy Training Pipeline to AI Platform Pipelines

The purpose of this notebook is to compile and deploy the TFX pipeline to AI Platform Pipelines. You need to setup [AI Platform Pipelines](https://cloud.google.com/ai-platform/pipelines/docs/setting-up) as a prerequisite, noting the **GKE Cluster Name**, **GKE Cluster Zone**, and the **GKE Namespace** of the Kubeflow pipelines installation. The notebook covers the following tasks:
1. Set pipeline configurations
2. Build Container Image
3. Compile TFX Pipeline
4. Deploy the pipeline to AI Platform Pipelines (Hosted KFP).

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf
import logging

logging.getLogger().setLevel(logging.INFO)

In [ ]:
PROJECT = 'ksalama-cloudml' # Change to your project Id.
REGION = 'us-central1'
BUCKET = 'ksalama-cloudml-us' # Change to your bucket.

## 1. Set pipeline configurations

In [ ]:
os.environ["DATASET_DISPLAY_NAME"] = 'chicago_taxi_tips'
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/ucaip_demo/chicago_taxi/pipelines_kfp_runner"
os.environ["TRAIN_LIMIT"] = "85000"
os.environ["TEST_LIMIT"] = "15000"
os.environ["BEAM_RUNNER"] = "DataflowRunner"
os.environ["TRAINING_RUNNER"] = "caip"

os.environ["GKE_CLUSTER_NAME"] = "ksalama-cloudml-dev-01" # Change to your GKE cluser name.
os.environ["GKE_CLUSTER_ZONE"] = "europe-west2-a" # Change to your GKE cluser zone.
os.environ["NAMESPACE"] = "kubeflow-pipelines" # Change to your KFP pipeline nsamespace.

In [ ]:
from tfx_pipeline import config
for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

## 2. Build Container Image

In [ ]:
!echo $IMAGE_URI

In [ ]:
!gcloud builds submit --tag $IMAGE_URI . --timeout=15m

## 3. Compile TFX Pipeline

In [ ]:
!rm $PIPELINE_NAME.tar.gz

In [ ]:
!tfx pipeline compile \
    --engine=kubeflow \
    --pipeline_path=tfx_pipeline/kfp_runner.py

## 4. Deploy the Compiled Pipeline to KFP

In [ ]:
%%bash

gcloud container clusters get-credentials ${GKE_CLUSTER_NAME} --zone ${GKE_CLUSTER_ZONE}
export KFP_ENDPOINT=$(kubectl describe configmap inverse-proxy-config -n ${NAMESPACE} | grep "googleusercontent.com")

kfp --namespace=${NAMESPACE} --endpoint=${KFP_ENDPOINT} \
    pipeline upload \
    --pipeline-name=${PIPELINE_NAME} \
    ${PIPELINE_NAME}.tar.gz

In [ ]:
!echo $(kubectl describe configmap inverse-proxy-config -n ${NAMESPACE} | grep "googleusercontent.com")

![Pipeline execution](kfp-pipeline.png)